# Building the Model

In [71]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Softmax
from keras.models import Model

encoding=16

input_level=Input(shape=(10,10,7))
encoded_level=Input(shape=(encoding,))

x = Conv2D(64, (3,3), activation="relu", padding="same")(input_level)
x = MaxPooling2D((2,2), padding="same")(x)
x = Conv2D(32, (3,3), activation="relu", padding="same")(x)
x = MaxPooling2D((4,4), padding="valid")(x)
x = Dense(32, activation="relu")(x)
x = Dense(encoding, activation="sigmoid")(x)
encoded = Flatten()(x)

x = Reshape((1,1,16))(encoded)
x = Dense(32, activation="relu")(x)
x = UpSampling2D((4,4))(x)
x = Conv2D(32, (3,3), activation="relu", padding="same")(x)
x = UpSampling2D((2,2))(x)
x = Conv2D(64, (3,3), activation="relu", padding="valid")(x)
x = UpSampling2D((2,2))(x)
x = Conv2D(7, (3,3), padding="valid")(x)
decoded = Softmax(axis=3)(x)

autoencoder = Model(input_level, decoded)
encoder = Model(input_level, encoded)
decoded = autoencoder.layers[-9](encoded_level)
decoded = autoencoder.layers[-8](decoded)
decoded = autoencoder.layers[-7](decoded)
decoded = autoencoder.layers[-6](decoded)
decoded = autoencoder.layers[-5](decoded)
decoded = autoencoder.layers[-4](decoded)
decoded = autoencoder.layers[-3](decoded)
decoded = autoencoder.layers[-2](decoded)
decoded = autoencoder.layers[-1](decoded)
decoder = Model(encoded_level,decoded)

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# Loading Data

In [83]:
import numpy as np

data = np.load("FinalData/original_data.npz")
inputData,outputData=data["input"],data["output"]
np.random.shuffle(inputData)
np.random.shuffle(outputData)
trainInput,trainOutput=inputData[0:15000],outputData[0:15000]
testInput,testOutput=inputData[15000:],outputData[15000:]

# Helper Functions

In [97]:
import numpy as np

def numpyToString(array):
    gameCharacters="# @$.+*"
    char_to_int = dict((c, i) for i, c in enumerate(gameCharacters))
    int_to_char = dict((i, c) for i, c in enumerate(gameCharacters))
    intArray = np.argmax(array, axis=2)
    output=""
    for (i,j), index in np.ndenumerate(intArray):
        if i > 0 and j == 0:
            output += "\n"
        output += int_to_char[index]
    return output